# Inserting to the MySQL database example

In [1]:
import pymysql.cursors
from pymysql.cursors import DictCursor
from datetime import datetime
from os import environ as ENV
from dotenv import load_dotenv

## Connect to the database

In [2]:
load_dotenv()

True

In [5]:
conn = pymysql.connect(host=ENV['DB_HOST'],
               user=ENV['DB_USER'],
               password=ENV['DB_PASSWORD'],
               database=ENV['DB_NAME'],
               port=int(ENV['DB_PORT']),
               cursorclass=DictCursor)

## Inserting a set of values into the transaction table

In [18]:
datetime_obj = datetime.strptime("2025-01-13 09:21:00", "%Y-%m-%d %H:%M:%S")
with conn:
    with conn.cursor() as cur:
        query = """
            INSERT INTO FACT_Transaction 
                (transaction_id,
                truck_id,
                payment_method_id,
                total,
                at)
            VALUES 
                (%s, %s, %s, %s, %s);
        """
        cur.execute(query, (2, 4, 1, 10.7, datetime_obj))
    conn.commit()

## Checking insertion and fetch

In [6]:
with conn:
    with conn.cursor() as cur:
        query = "SELECT * FROM transaction_info ORDER BY transaction_id ASC LIMIT 20;"
        cur.execute(query)
        result = cur.fetchall()
    for row in result:
        print(row)

{'transaction_id': 1, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('7.00'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 6)}
{'transaction_id': 2, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('8.40'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 11)}
{'transaction_id': 3, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('7.00'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 21)}
{'transaction_id': 4, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('7.00'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 36)}
{'transaction_id': 5, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('7.00'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 44)}
{'transaction_id': 6, 'truck_id': 1, 'truck_name': 'Burrito Madness', 'total': Decimal('10.60'), 'payment_method': 'card', 'at': datetime.datetime(2025, 1, 13, 9, 59)